In [18]:
!pip install --upgrade --user google-cloud-aiplatform google-cloud-storage
!pip install langchain_community
!pip install google-cloud-aiplatform
!pip install google-cloud-storage
!pip install langchain_community
!pip install langchain
!pip install pypdf
!pip install google-cloud-firestore
!pip install textract
!pip install PyPDF2
!pip install pandas
!pip install firebase_admin

In [1]:
import IPython
from IPython.display import Markdown, display
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [21]:
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"

# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")
PROJECT_ID

'qwiklabs-gcp-02-0c71a0fc92ea'

In [22]:
#enable apis
!gcloud services enable \
  aiplatform.googleapis.com \
  storage.googleapis.com \
  run.googleapis.com \
  compute.googleapis.com \
  cloudresourcemanager.googleapis.com \
  container.googleapis.com \
  artifactregistry.googleapis.com \
  containerregistry.googleapis.com \
  containerscanning.googleapis.com

Operation "operations/acat.p2-304935860940-a917112d-a622-446e-aef7-5b537c800a0b" finished successfully.


In [27]:
#create firestore db
#!gcloud config set project PROJECT_ID
!gcloud firestore databases create \
  --project=PROJECT_ID \
  --location=LOCATION \
  --type=firestore-native

metadata:
  '@type': type.googleapis.com/google.firestore.admin.v1.CreateDatabaseMetadata
name: projects/qwiklabs-gcp-02-0c71a0fc92ea/databases/(default)/operations/8ePlyiwlrrxWSsI4o_8M_BAqMWxhcnRuZWMtc3ULIgUQAvXstbAQBrGfz-UIDAovGg
response:
  '@type': type.googleapis.com/google.firestore.admin.v1.Database
  appEngineIntegrationMode: DISABLED
  concurrencyMode: PESSIMISTIC
  createTime: '2024-04-23T16:55:01.784014Z'
  deleteProtectionState: DELETE_PROTECTION_DISABLED
  earliestVersionTime: '2024-04-23T16:55:01.784014Z'
  etag: IPy+6NHm2IUDMM7z59Hm2IUD
  locationId: us-central1
  name: projects/qwiklabs-gcp-02-0c71a0fc92ea/databases/(default)
  pointInTimeRecoveryEnablement: POINT_IN_TIME_RECOVERY_DISABLED
  type: FIRESTORE_NATIVE
  uid: fc0cffa3-38c2-4a56-bcae-252ccae5e3f1
  updateTime: '2024-04-23T16:55:01.784014Z'
  versionRetentionPeriod: 3600s


In [28]:
def create_data_packet(file_name, file_type, page_number, file_content):
    """Creating a simple dictionary to store all information (content and metadata)
    extracted from the document"""
    data_packet = {}
    data_packet["file_name"] = file_name
    data_packet["file_type"] = file_type
    data_packet["page_number"] = page_number
    data_packet["content"] = file_content
    return data_packet

In [ ]:
# if needed to load from cloud storage
# !mkdir test
# !gsutil -m cp -r gs://qwiklabs-gcp-00-c908ca7e2977/fpc-manual.pdf .

In [29]:
#pdf parser
import os
import textract
from PyPDF2 import PdfReader

final_data = []

file_name = 'fpc-manual'
file_type ='pdf'

path = f"fpc-manual.pdf"

# loading pdf files, with page numbers as metadata.
reader = PdfReader(path)
for i, page in enumerate(reader.pages):

    text = page.extract_text()
    if text:
        packet = create_data_packet(
              file_name, file_type, page_number=int(i + 1), file_content=text
        )

        final_data.append(packet)

In [30]:
# converting the data that has been read from GCS to Pandas DataFrame for easy readibility and downstream logic
import pandas as pd
pdf_data = pd.DataFrame.from_dict(final_data)
pdf_data.reset_index(inplace=True, drop=True)
pdf_data.head()

,file_name,file_type,page_number,content
0,fpc-manual,pdf,1,The Health Code\nThese are regulations that we...
1,fpc-manual,pdf,2,The United States has one of the\nsafest food ...
2,fpc-manual,pdf,3,What is Ready-To- Eat Food?\nAny food product ...
3,fpc-manual,pdf,4,Ice-Point Method\n/H20321/H20321Fill a contain...
4,fpc-manual,pdf,5,Fresh fish\nThere is no inspection for fresh\n...


In [31]:
# init the vertexai package
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [32]:
# Load the text embeddings model
from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko@002")

In [33]:
#batch embeddings to embedding api
import time
import tqdm  # to show a progress bar

# get embeddings for a list of texts
BATCH_SIZE = 5


def get_embeddings_wrapper(texts):
    embs = []
    for i in tqdm.tqdm(range(0, len(texts), BATCH_SIZE)):
        time.sleep(1)  # to avoid the quota error
        result = model.get_embeddings(texts[i : i + BATCH_SIZE])
        embs = embs + [e.values for e in result]
    return embs


In [34]:
# get embeddings for the question titles and add them as "embedding" column
pdf_data = pdf_data.assign(embedding=get_embeddings_wrapper(list(pdf_data.content)))
pdf_data.head()

100%|██████████| 19/19 [00:23<00:00,  1.24s/it]


,file_name,file_type,page_number,content,embedding
0,fpc-manual,pdf,1,The Health Code\nThese are regulations that we...,"[-0.013569382950663567, -0.033456653356552124,..."
1,fpc-manual,pdf,2,The United States has one of the\nsafest food ...,"[-0.01585172675549984, -0.012489386834204197, ..."
2,fpc-manual,pdf,3,What is Ready-To- Eat Food?\nAny food product ...,"[-0.011286059394478798, -0.011021223850548267,..."
3,fpc-manual,pdf,4,Ice-Point Method\n/H20321/H20321Fill a contain...,"[-0.013480892404913902, -0.017172927036881447,..."
4,fpc-manual,pdf,5,Fresh fish\nThere is no inspection for fresh\n...,"[0.0021019037812948227, -0.018236981704831123,..."


In [36]:
#load data to firestore db
import pandas as pd
import firebase_admin
from firebase_admin import firestore

# Initialize the Firestore client
firebase_admin.initialize_app()
db = firestore.client()

# Get the PDF data
#pdf_data = pd.read_csv("pdf_data.csv", index_col=0)

# Iterate over the rows in the PDF data
for index, row in pdf_data.iterrows():
    # Get the page number
    page_number = row["page_number"]

    # Get the content and embedding
    content = row["content"]
    embedding = row["embedding"]

    # Create a document reference
    doc_ref = db.collection("pages").document(str(page_number))

    # Set the document data
    doc_ref.set({
        "content": content,
        "embedding": embedding
    })


In [37]:
# #create a embedding.jsonl file
# import pandas as pd
# import json

# # Get the PDF data
# #pdf_data = pd.read_csv("pdf_data.csv", index_col=0)

# # Create a list of dictionaries, where each dictionary contains the page number and embedding
# embeddings = []
# for index, row in pdf_data.iterrows():
#     embeddings.append({
#         "page_number": row["page_number"],
#         "embedding": row["embedding"]
#     })

# # Convert the list of dictionaries to JSON-L format
# json_l = "\n".join(json.dumps(embedding) for embedding in embeddings)

# # Write the JSON-L data to a file
# with open("embeddings.jsonl", "w") as f:
#     f.write(json_l)

In [44]:
#save the embeddings.jsonl file to cloud storage
# from google.cloud import storage

# # Create a storage client
# storage_client = storage.Client()

# # Get the bucket
# bucket = storage_client.bucket("qwiklabs-gcp-02-4b2f76b52489")

# # Upload the file
# bucket.blob("embeddings.json").upload_from_filename("embeddings.jsonl")

# # Print a success message
# print("File uploaded successfully to Google Cloud Storage.")

In [39]:
# init the aiplatform package
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

BUCKET_URI = "gs://"+PROJECT_ID+"/embeddings.jsonl"

In [40]:
# create index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"assessment-index",
    contents_delta_uri=BUCKET_URI,
    dimensions=768,
    approximate_neighbors_count=5,
)


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Creating MatchingEngineIndex
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Create MatchingEngineIndex backing LRO: projects/304935860940/locations/us-central1/indexes/5395826925031653376/operations/7756736259703177216
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex created. Resource name: projects/304935860940/locations/us-central1/indexes/5395826925031653376
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:To use this MatchingEngineIndex in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:index = aiplatform.MatchingEngineIndex('projects/304935860940/locations/us-central1/indexes/5395826925031653376')


In [41]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"assessment-index-endpoint", public_endpoint_enabled=True
)


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Creating MatchingEngineIndexEndpoint
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Create MatchingEngineIndexEndpoint backing LRO: projects/304935860940/locations/us-central1/indexEndpoints/4121589705462513664/operations/7128484111684993024
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint created. Resource name: projects/304935860940/locations/us-central1/indexEndpoints/4121589705462513664
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:To use this MatchingEngineIndexEndpoint in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/304935860940/locations/us-central1/indexEndpoints/4121589705462513664')


In [42]:
#DEPLOYED_INDEX_ID = f"assessmentindexendpoint"
DEPLOYED_INDEX_ID =  f"assessment_deployed1"
print(DEPLOYED_INDEX_ID)

assessment_deployed1


In [43]:
# deploy the Index to the Index Endpoint

my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/304935860940/locations/us-central1/indexEndpoints/4121589705462513664
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/304935860940/locations/us-central1/indexEndpoints/4121589705462513664/operations/6045368401302388736
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/304935860940/locations/us-central1/indexEndpoints/4121589705462513664


resource name: projects/304935860940/locations/us-central1/indexEndpoints/4121589705462513664

In [45]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
    index_endpoint_name ="projects/304935860940/locations/us-central1/indexEndpoints/4121589705462513664"
)

In [ ]:
# def text_embedding(text_to_embed) -> list:
#     """Text embedding with a Large Language Model."""
#     model = TextEmbeddingModel.from_pretrained("textembedding-gecko@002")
#     embeddings = model.get_embeddings(text_to_embed)
#     for embedding in embeddings:
#         vector = embedding.values
#         #print(f"Length of Embedding Vector: {len(vector)}")
#     return vector

# #query the vector database
# #question =

# from vertexai.language_models import  TextEmbeddingInput

# QUESTION="What is the minimum safe cooking temperature for chicken?"

# question_with_task_type = TextEmbeddingInput(
#     text=QUESTION,
#     task_type='RETRIEVAL_QUERY'
# )



In [ ]:
#question_embeddings = get_embeddings_wrapper(question_with_task_type)
# question_embeddings = text_embedding([question_with_task_type])

In [ ]:

#question_embeddings

In [ ]:

# run query
# response = my_index_endpoint.find_neighbors(
#     deployed_index_id = DEPLOYED_INDEX_ID,
#     queries = [question_embeddings],
#     num_neighbors = 10
# )


In [ ]:
# response

[[]]

In [ ]:
# # show the results
# for idx, neighbor in enumerate(response[0]):
#     print(f"{neighbor.distance:.2f} {neighbor.id}")


In [ ]:
# documents = {}
# for index, neighbor in enumerate(response[0]):
#     id = str(neighbor.id)
#     document = db.collection(collection_name).document(id).get()
#     documents.append(document.to_dict["pages"])

In [ ]:
# pages = "\n\n".join(documents)
# print(len(pages))
# pages

0


''

In [ ]:
# # show the result
# import numpy as np

# for idx, neighbor in enumerate(response[0]):
#     id = np.int64(neighbor.id)
#     similar = df.query("id == @id", engine="python")
#     print(f"{neighbor.distance:.4f} {similar.title.values[0]}")

In [ ]:
#  documents = {}
#     for index, neighbor in enumerate(respone[0]):
#         id = str(neighbor.id)
#         document = db.collection(collection_name).document(id).get()
#         documents.append(document.to_dict["page"])


#     pages = "\n\n".join(documents)

In [ ]:
# create a firestore database for the pages